In [170]:
#Import libraries
import json
from urllib.request import urlopen
import ssl
import requests
import pandas as pd
import folium
import re

In [171]:
#import AED waiting data json
context = ssl._create_unverified_context()
link = urlopen("http://www.ha.org.hk/opendata/aed/aedwtdata-en.json", context=context)
aedwait_json = json.loads(link.read())

In [172]:
#Finding related information about waiting data
hospital = []
waitingTime = []
for i in range(len(aedwait_json['waitTime'])):
    hospital.append(aedwait_json['waitTime'][i]['hospName'])
    waitingTime.append(aedwait_json['waitTime'][i]['topWait'])

waitingList = pd.DataFrame({'Hospital':hospital, 'Waiting Time': waitingTime})
updateTime = aedwait_json['updateTime']

In [173]:
#View data
waitingList

,Hospital,Waiting Time
0,Alice Ho Miu Ling Nethersole Hospital,Over 2 hours
1,Caritas Medical Centre,Over 2 hours
2,Kwong Wah Hospital,Over 2 hours
3,North District Hospital,Over 6 hours
4,North Lantau Hospital,Over 2 hours
5,Princess Margaret Hospital,Over 3 hours
6,Pok Oi Hospital,Over 2 hours
7,Prince of Wales Hospital,Over 8 hours
8,Pamela Youde Nethersole Eastern Hospital,Over 2 hours
9,Queen Elizabeth Hospital,Over 2 hours


In [174]:
#Import Geospatial data of hospitals
link2 = urlopen("https://www.ha.org.hk/opendata/facility-hosp.json", context=context)
hospitalLoc = pd.read_json(link2)


In [175]:
#Join both sources
hospitalLoc['institution_eng'] = hospitalLoc['institution_eng'].str.replace('.', '')
full_list = hospitalLoc[['institution_eng', 'latitude', 'longitude', 'address_eng']].merge(waitingList, left_on = 'institution_eng', right_on = 'Hospital')
full_list.drop(labels = 'institution_eng', axis=1, inplace=True)

In [176]:
#View combined data
full_list

,latitude,longitude,address_eng,Hospital,Waiting Time
0,22.269180,114.23643,"3 Lok Man Road, Chai Wan, HK",Pamela Youde Nethersole Eastern Hospital,Over 2 hours
1,22.275909,114.17529,"266 Queen's Road East, Wan Chai, HK",Ruttonjee Hospital,Over 2 hours
2,22.208059,114.03151,"Cheung Chau Hospital Road, Tung Wan, Cheung Chau",St John Hospital,Around 1 hour
3,22.270400,114.13117,"102 Pokfulam Road, HK",Queen Mary Hospital,Over 3 hours
4,22.315174,114.17244,"25 Waterloo Road, KLN",Kwong Wah Hospital,Over 2 hours
5,22.308860,114.17519,"30 Gascoigne Road, KLN",Queen Elizabeth Hospital,Over 2 hours
6,22.317964,114.27021,"No. 2 Po Ning Lane, Hang Hau, Tseung Kwan O",Tseung Kwan O Hospital,Over 3 hours
7,22.322291,114.22790,"130 Hip Wo Street, Kwun Tong, KLN",United Christian Hospital,Over 8 hours
8,22.340629,114.15231,"111 Wing Hong Street, Sham Shui Po, KLN",Caritas Medical Centre,Over 2 hours
9,22.282571,113.93914,"8 Chung Yan Road, Tung Chung, Lantau Island",North Lantau Hospital,Over 2 hours


In [177]:
# Import GOPC list
link3 = urlopen("https://www.ha.org.hk/opendata/facility-gop.json", context=context)
gopcLoc = pd.read_json(link3)

In [178]:
gopcList = gopcLoc[['institution_eng', 'latitude', 'longitude', 'address_eng']]

In [179]:
#Colouring of dots by time. 
#Since it is normally expected non-urgent cases will be consulted in 4-6 hours, it will be cut offs of colours.
def grading(string):
    try:
        weight = int(re.findall(r'\d', string)[0])
    except:
        return 'gray'
    if weight < 8 :
        return 'blue'
    else:
        return 'red'

In [180]:
#Generate a Hong Kong map using folium
hkmap = folium.Map(width=1000,height=480, location=[22.36, 114.15], zoom_start=11, tiles="Cartodb Positron")

In [181]:
#Putting dots and titles

#Add dots to AEDs
r = folium.FeatureGroup(name= '<span style="color:red">AED, waiting time more than 8 hours</span>')
b = folium.FeatureGroup(name='<span style="color:blue">AED, waiting time less than 8 hours</span>')
for i in full_list.iterrows():
    j = i[1]
    color=grading(j['Waiting Time'])
    circle = folium.CircleMarker(
    location=[j['latitude'], j['longitude']],
    radius=10,
    color=color,
    stroke=True,
    fill=True,
    fill_opacity=0.3,
    opacity=1,
    popup = folium.map.Popup(html = '{} <br> {}'.format(j['Hospital'], j['address_eng']), max_width = 200),
    tooltip='{}, {}'.format(j['Hospital'], j['Waiting Time']),
)
    if color == 'blue':
        b.add_child(circle)
    elif color == 'red':
        r.add_child(circle)
    else:
        circle.add_to(hkmap)
b.add_to(hkmap)
r.add_to(hkmap)

    
#Add dots to GOPC
g = folium.FeatureGroup(name='<span style="color:green">General Outpatient Clinics</span>')
for i in gopcList.iterrows():
    j = i[1]
    circle = folium.CircleMarker(
    location=[j['latitude'], j['longitude']],
    radius=3,
    color='green',
    stroke=False,
    fill=True,
    fill_opacity=0.8,
    opacity=1,
    popup = folium.map.Popup(html = '{} <br> {}'.format(j['institution_eng'], j['address_eng']), max_width = 200),
    tooltip='{}'.format(j['institution_eng'])
)
    g.add_child(circle)
g.add_to(hkmap)
    

#Add Title
loc = 'AED Longest Waiting Time as at {}'.format(updateTime)
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   
hkmap.get_root().html.add_child(folium.Element(title_html))

#
#hkmap.add_child(r)
#hkmap.add_child(b)

folium.map.LayerControl('topleft', collapsed= False).add_to(hkmap)

In [182]:
#Show the map
hkmap